# DPU: DAG Processing Unit for probabilistic ML and sparse matrix algebra 
### (Participating in category 1)

```
KU Leuven MICAS Team
SPDX-License-Identifier: Apache-2.0
```

|Name|Affiliation|IEEE Member|SSCS Member|
|:--:|:----------:|:----------:|:----------:|
|Nimish Shah (Leader)|PhD researcher, KU Leuven, Belgium|Yes|Yes|
|Prof. Marian Verhelst (Supervisor)|Professor, KU Leuven, Belgium|Yes|Yes|


This notebook implements the silicon proven DAG Processing Unit (DPU) in SKY130 tech using open-source tools.
The original DPU is prototyped on chip in a commercial 28nm tech and published in [ISSCC'21](https://ieeexplore.ieee.org/abstract/document/9366061) and [JSSC'22](https://ieeexplore.ieee.org/document/9663412/metrics#metrics).

DPU is a specialized digital processor designed for irregularly structured data flow graphs (DFG) from probabilistic ML and sparse linear algebra workloads. The irregular DFG structure poses challenges to parallel execution leading to low throughput and energy efficiency on CPUs and GPUs. Some of these parallelization challenges include frequent synchronization and communication between cores, and irregular memory accesses with poor locality. The salient DPU features to address these challenges and further improve energy efficiency are as follows:
   
 * Hardware-supported synchronization primitives for parallel cores.
 * A software-managed scratchpad-based memory hierarchy to minimize the impact of accesses with poor locality.
 * Decoupled memory and computational instruction streams to hide the long memory latency caused by bank conflicts due to irregular accesses.
 * A novel data representation based on [posit](http://www.johngustafson.net/pdfs/BeatingFloatingPoint.pdf)<sup>TM</sup> suitable for the target workloads, along with a precision-scalable arithmetic hardware that can perform 1x32, 2x16, or 4x8 posit operations depending on the application need.

These features of DPU achieve orders of magnitude higher throughput and energy efficiency over CPUs and GPUs for the irregular target workloads.

**_Index Terms:_** Custom digital processor, irregular data flow graphs, open source, end-to-end flow, probabilistic machine learning, sparse linear algebra, OpenLANE, OpenRAM, posit arithmetic

Citations: 

N. Shah, L. I. G. Olascoaga, S. Zhao, W. Meert and M. Verhelst, "9.4 PIU: A 248GOPS/W Stream-Based Processor for Irregular Probabilistic Inference Networks Using Precision-Scalable Posit Arithmetic in 28nm," 2021 IEEE International Solid- State Circuits Conference (ISSCC), 2021, pp. 150-152, doi: 10.1109/ISSCC42613.2021.9366061.

N. Shah, L. I. G. Olascoaga, S. Zhao, W. Meert and M. Verhelst, "DPU: DAG Processing Unit for Irregular Graphs With Precision-Scalable Posit Arithmetic in 28 nm," in IEEE Journal of Solid-State Circuits (JSSC), vol. 57, no. 8, pp. 2586-2596, Aug. 2022, doi: 10.1109/JSSC.2021.3134897.



## DPU architecture

The following figure shows the DPU architecture designed for acyclic DFGs from probabilistic ML and sparse matrix operations. The microarchitecture of major blocks of DPU are shown in detail below. This notebook implements the DPU with 2 parallel compute units.

![DPU architecture](./images/arch.png)

## Design flow
This notebook performs the physical implementation of DPU in SKY130 using only open-source CAD tool:
*   The OpenLANE script flow which uses:
    *   Yosys for logic synthesis
    *   Openroad for placing and routing
    *   Klayout to produce the final GDS file
    *   Magic for DRC, LVS and PEX.
*   OpenRAM to generate SRAM macros

Set up a conda enviornment and install SKY130 PDK, OpenLANE, and other required tools.

In [1]:
!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj bin/micromamba
import os
import pathlib
import sys

conda_prefix_path = pathlib.Path('conda-env')
site_package_path = conda_prefix_path / 'lib/python3.7/site-packages'
sys.path.append(str(site_package_path.resolve()))
CONDA_PREFIX = str(conda_prefix_path.resolve())
PATH = os.environ['PATH']
LD_LIBRARY_PATH = os.environ.get('LD_LIBRARY_PATH', '')
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}
%env LD_LIBRARY_PATH={CONDA_PREFIX}/lib:{LD_LIBRARY_PATH}

!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!bin/micromamba install --yes --prefix $CONDA_PREFIX \
                        --channel litex-hub \
                        --channel main \
                        open_pdks.sky130a \
                        magic \
                        openroad \
                        netgen \
                        yosys \
                        openlane
!bin/micromamba install --yes --prefix $CONDA_PREFIX \
                        --channel conda-forge \
                        tcllib gdstk pyyaml click pandas svgutils pip

bin/micromamba
env: CONDA_PREFIX=/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/conda-env
env: PATH=/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/conda-env/bin:/users/micas/nshah/.sdkman/candidates/sbt/current/bin:/users/micas/nshah/.sdkman/candidates/java/current/bin:/users/micas/nshah/no_backup/.pyenv/plugins/pyenv-virtualenv/shims:/users/micas/nshah/no_backup/.pyenv/shims:/users/micas/nshah/no_backup/.pyenv/bin:/users/micas/nshah/no_backup/.pyenv/versions/3.7.7/bin:/volume1/users/nshah/anaconda3/anaconda3/envs/sscs_final/bin:/volume1/users/nshah/anaconda3/anaconda3/condabin:/esat/puck1/users/nshah/cpu_gpu_parallel/intel_full_compiler/intel/mkl/2021.1-beta10/bin/intel64:/esat/puck1/users/nshah/cpu_gpu_parallel/intel_full_compiler/intel/advisor/2021.1-beta10/bin64:/esat/puck1

main/noarch        ━━━╸━━━━━━━━━━━━━━━╸━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.3s[+] 0.4s
litex-hub/linux-64 ━━━━━━━━━━━━╸━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.4s
litex-hub/noarch   ━━━━━━━━━━━━━━━╸━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.4s
main/linux-64      ━━╸━━━━━━━━━━━━━━━╸━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.4s
main/noarch        ━━━━╸━━━━━━━━━━━━━━━╸━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.4slitex-hub/linux-64                                            No change
main/linux-64                                                 No change
litex-hub/noarch                                              No change
main/noarch                                                   No change

Pinned packages:
  - python 3.7*


Transaction

  Prefix: /users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/conda-env

  Updating specs:

   - open_pdks.sky130a
   - magic
   - openroad
   - netge

Linking libxml2-2.9.14-h74e7548_0
Linking libpng-1.6.37-hbc83047_0
Linking zstd-1.5.2-ha4553b6_0
Linking tk-8.6.12-h1ccaba5_0
Linking glib-2.69.1-h4ff587b_1
Linking sqlite-3.39.3-h5082296_0
Linking freetype-2.12.1-h4a9f257_0
Linking libboost-1.73.0-h28710b8_12
Linking python-3.7.15-haa1d7c7_0
Linking fontconfig-2.13.1-hef1e5e3_1
Linking certifi-2022.9.24-py37h06a4308_0
Linking cairo-1.16.0-h19f5f5c_2
Linking setuptools-65.5.0-py37h06a4308_0
Linking tcllib-1.20-0
Linking magic-8.3.341_0_gf066844-20221104_084554
Linking wheel-0.37.1-pyhd3eb1b0_0
Linking pip-22.2.2-py37h06a4308_0
Linking zipp-3.8.0-py37h06a4308_0
Linking typing_extensions-4.3.0-py37h06a4308_0
Linking pyyaml-6.0-py37h7f8727e_1
Linking importlib-metadata-4.11.3-py37h06a4308_0
Linking click-8.0.4-py37h06a4308_0
Linking yosys-0.23_6_g853f4bb3c-20221104_084554_py37
Linking netgen-1.5.242_0_g4edaf08-20221104_084554
Linking openroad-2.0_5614_g71fa4b775-20221104_084554
Linking openlane-2022.11.12_3_g1298859-20221104_084554
Transa

### OpenRAM compiler
Download the OpenRAM compiler and install the prerequisites.

In [2]:
!git clone -b v1.1.19 https://github.com/VLSIDA/OpenRAM.git
!which pip3
!pip3 install -r OpenRAM/requirements.txt

Cloning into 'OpenRAM'...
remote: Enumerating objects: 33779, done.
remote: Counting objects: 100% (7383/7383), done.
remote: Compressing objects: 100% (1856/1856), done.
remote: Total 33779 (delta 5604), reused 7180 (delta 5426), pack-reused 26396
Receiving objects: 100% (33779/33779), 55.05 MiB | 19.86 MiB/s, done.
Resolving deltas: 100% (26978/26978), done.
Note: checking out 'f66aac3264598eeae31225c62b6a4af52412d407'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b new_branch_name

/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/conda-env/bin/pip3


Set up the enviornment for OpenRAM and OpenLANE. 

In [3]:
import os
import pathlib
import sys

openram_root_path = pathlib.Path('OpenRAM')
OPENRAM_ROOT= str(openram_root_path.resolve())
OPENLANE_ROOT=CONDA_PREFIX + '/share/openlane'
PATH=os.environ['PATH']
%env OPENRAM_ROOT={OPENRAM_ROOT}
%env OPENRAM_HOME={OPENRAM_ROOT}/compiler
%env OPENRAM_TECH={OPENRAM_ROOT}/technology/sky130
%env PYTHONPATH={OPENRAM_ROOT}/compiler:{OPENRAM_ROOT}/technology:{OPENRAM_ROOT}/technology/sky130/modules
%env PDK_ROOT={CONDA_PREFIX}/share/pdk
%env PDK=sky130A
%env STD_CELL_LIBRARY=sky130_fd_sc_hd
%env STD_CELL_LIBRARY_OPT=sky130_fd_sc_hd
%env TCLLIBPATH={CONDA_PREFIX}/lib/tcllib1.20
%env OPENLANE_ROOT={OPENLANE_ROOT}
%env PATH={PATH}:{OPENLANE_ROOT}:{OPENLANE_ROOT}/scripts
%env OPENLANE_LOCAL_INSTALL=1
!make -C OpenRAM SRAM_GIT_REPO=https://github.com/google/skywater-pdk-libs-sky130_fd_bd_sram.git

env: OPENRAM_ROOT=/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM
env: OPENRAM_HOME=/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler
env: OPENRAM_TECH=/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/technology/sky130
env: PYTHONPATH=/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler:/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/technology:/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_I

‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/openram_dp_cell_replica/sky130_fd_bd_sram__openram_dp_cell_replica.gds’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__openram_dp_cell_replica.gds’
‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/openram_dp_nand2_dec/sky130_fd_bd_sram__openram_dp_nand2_dec.gds’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__openram_dp_nand2_

‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_blkinv_mcon/sky130_fd_bd_sram__sram_dp_blkinv_mcon.gds’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_blkinv_mcon.gds’
‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_blkinv_met23/sky130_fd_bd_sram__sram_dp_blkinv_met23.gds’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_blkinv_met23.gds’
‘/u

‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_cell_opt1a_poly_siz/sky130_fd_bd_sram__sram_dp_cell_opt1a_poly_siz.gds’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_cell_opt1a_poly_siz.gds’
‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_cell_opt2/sky130_fd_bd_sram__sram_dp_cell_opt2.gds’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_cel

‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_colend_li_drop/sky130_fd_bd_sram__sram_dp_colend_li_drop.gds’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_colend_li_drop.gds’
‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_colend_opt1/sky130_fd_bd_sram__sram_dp_colend_opt1.gds’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/gds_lib/sky130_fd_bd_sram__sram_dp_colend_opt1.gd

...

‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_swldrv_opt3di/sky130_fd_bd_sram__sram_dp_swldrv_opt3di.maglef’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_swldrv_opt3di.mag’
‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_swldrv_opt4a/sky130_fd_bd_sram__sram_dp_swldrv_opt4a.maglef’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_swl

‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_wls_half/sky130_fd_bd_sram__sram_dp_wls_half.maglef’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_wls_half.mag’
‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_dp_wls_p1m_ser/sky130_fd_bd_sram__sram_dp_wls_p1m_ser.maglef’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_dp_wls_p1m_ser.mag’
‘/u

‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_cell_metopt1/sky130_fd_bd_sram__sram_sp_cell_metopt1.maglef’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_sp_cell_metopt1.mag’
‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_cell_opt1/sky130_fd_bd_sram__sram_sp_cell_opt1.maglef’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_sp_cell_opt1.ma

‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_colenda_ce/sky130_fd_bd_sram__sram_sp_colenda_ce.maglef’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_sp_colenda_ce.mag’
‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_colenda_cent/sky130_fd_bd_sram__sram_sp_colenda_cent.maglef’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_sp_colenda_cent

‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_wlstrap_p/sky130_fd_bd_sram__sram_sp_wlstrap_p.maglef’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_sp_wlstrap_p.mag’
‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/sky130_fd_bd_sram/cells/sram_sp_wlstrap_p1m_siz/sky130_fd_bd_sram__sram_sp_wlstrap_p1m_siz.maglef’ -> ‘/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/OpenRAM/compiler/../technology/sky130/maglef_lib/sky130_fd_bd_sram__sram_sp_wlstrap_p

### SRAM specification
Write configuration files with the required SRAM specification. This will be consumed by the OpenRAM compiler.

In [4]:
%%writefile ram_config_32b.py
"""
Single ported RAM
"""
word_size = 32 # Bits
num_words = 256
human_byte_size = "{:.0f}kbytes".format((word_size * num_words)/1024/8)
write_size = word_size # Bits

# Single port
num_rw_ports = 1
num_r_ports = 0
num_w_ports = 0
num_spare_rows= 1 # required only in 1rw case
num_spare_cols= 1 # requires only for 1rw case
ports_human = '1rw'

tech_name = "sky130"
nominal_corner_only = True

route_supplies = "ring"
uniquify = True

output_name = f"sky130_sram_{ports_human}_{word_size}x{num_words}_{write_size}"
output_path = "./OpenRAM"

Writing ram_config_32b.py


In [5]:
%%writefile ram_config_24b.py
"""
Single ported RAM
"""
word_size = 24 # Bits
num_words = 128
human_byte_size = "{:.0f}kbytes".format((word_size * num_words)/1024/8)
write_size = word_size # Bits

# Single port
num_rw_ports = 1
num_r_ports = 0
num_w_ports = 0
num_spare_rows= 1 # required only in 1rw case
num_spare_cols= 1 # requires only for 1rw case
ports_human = '1rw'

tech_name = "sky130"
nominal_corner_only = True

route_supplies = "ring"
uniquify = True

output_name = f"sky130_sram_{ports_human}_{word_size}x{num_words}_{write_size}"
output_path = "./OpenRAM"

Writing ram_config_24b.py


### SRAM generation
Launch the OpenRAM compiler to generate required macros. This can take up to 2 hours.

In [6]:
!which python3
!python3 $OPENRAM_HOME/openram.py ram_config_32b.py
!python3 $OPENRAM_HOME/openram.py ram_config_24b.py

/users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/conda-env/bin/python3
|==============================================================================|
|=========                      OpenRAM v1.1.19                       =========|
|=========                                                            =========|
|=========               VLSI Design and Automation Lab               =========|
|=========        Computer Science and Engineering Department         =========|
|=========            University of California Santa Cruz             =========|
|=========                                                            =========|
|=========          Usage help: openram-user-group@ucsc.edu           =========|
|=========        Development help: openram-dev-group@ucsc.edu        =========|
|=========          Temp dir: /tmp/openram_nshah_95129_temp/          =========|
|=========     

** Submodules: 3.2 seconds
** Placement: 0.0 seconds
**** Retrieving pins: 0.0 seconds
**** Analyzing pins: 0.0 seconds
**** Finding blockages: 1.1 seconds
**** Converting blockages: 0.1 seconds
**** Converting pins: 0.1 seconds
**** Separating adjacent pins: 0.0 seconds
**** Enclosing pins: 0.0 seconds
*** Finding pins and blockages: 18.2 seconds
*** Maze routing pins: 56.5 seconds
**** Retrieving pins: 0.0 seconds
**** Analyzing pins: 0.5 seconds
**** Finding blockages: 3.6 seconds
**** Converting blockages: 0.1 seconds
**** Converting pins: 1.8 seconds
**** Separating adjacent pins: 1.2 seconds
**** Enclosing pins: 2.4 seconds
*** Finding pins and blockages: 28.8 seconds
*** Maze routing supplies: 104.2 seconds
** Routing: 296.3 seconds
** Verification: 0.0 seconds
** SRAM creation: 299.7 seconds
SP: Writing to /users/micas/nshah/Downloads/PhD/Academic/Bayesian_Networks_project/Hardware_Implementation/sscs-ose-code-a-chip.github.io/Notebooks/DAG_Processing_Unit/./OpenRAM/sky130_sram

### SRAM placement
Write a configuration file specifying the exact location and orientation of SRAM macros on the die. The instance names are obtained from the netlist after a syntheis run.

In [7]:
%%writefile macro_placement.cfg
FULL_DESIGN_WO_PERIPHERY_INS.GLOBAL_MEM_INS.mem_loop\[0\].SP_MEM_MODEL_INS.mem_32x256.sram                                                   315  100  S
FULL_DESIGN_WO_PERIPHERY_INS.GLOBAL_MEM_INS.mem_loop\[1\].SP_MEM_MODEL_INS.mem_32x256.sram                                                   1200 100  S
FULL_DESIGN_WO_PERIPHERY_INS.PROCESSING_LOGIC_INS.pe_loop\[0\].PE_TOP_INS.SP_MEM_MODEL_INS.mem_32x256.sram                                   100  500  FN
FULL_DESIGN_WO_PERIPHERY_INS.PROCESSING_LOGIC_INS.pe_loop\[1\].PE_TOP_INS.SP_MEM_MODEL_INS.mem_32x256.sram                                   1415 500  FN
FULL_DESIGN_WO_PERIPHERY_INS.STREAMS_TOP_INS.MULTIPLE_STREAMS_INSTR_INS.stream_loop\[0\].STREAM_INS.SP_MEM_MODEL_INS.genblk1.mem_24x128.sram 100  1400 N
FULL_DESIGN_WO_PERIPHERY_INS.STREAMS_TOP_INS.MULTIPLE_STREAMS_LD_INS.stream_loop\[0\].STREAM_INS.SP_MEM_MODEL_INS.genblk1.mem_24x128.sram    400  1400 N
FULL_DESIGN_WO_PERIPHERY_INS.STREAMS_TOP_INS.MULTIPLE_STREAMS_ST_INS.stream_loop\[0\].STREAM_INS.SP_MEM_MODEL_INS.genblk1.mem_24x128.sram    700  1400 N
FULL_DESIGN_WO_PERIPHERY_INS.STREAMS_TOP_INS.MULTIPLE_STREAMS_ST_INS.stream_loop\[1\].STREAM_INS.SP_MEM_MODEL_INS.genblk1.mem_24x128.sram    1050 1400 N
FULL_DESIGN_WO_PERIPHERY_INS.STREAMS_TOP_INS.MULTIPLE_STREAMS_LD_INS.stream_loop\[1\].STREAM_INS.SP_MEM_MODEL_INS.genblk1.mem_24x128.sram    1350 1400 N
FULL_DESIGN_WO_PERIPHERY_INS.STREAMS_TOP_INS.MULTIPLE_STREAMS_INSTR_INS.stream_loop\[1\].STREAM_INS.SP_MEM_MODEL_INS.genblk1.mem_24x128.sram 1650 1400 N

Writing macro_placement.cfg


### Configuration for OpenLANE
Specify the configuration for the synthesis and PnR flow.

In [11]:
%%writefile config.tcl

set ::env(STD_CELL_LIBRARY) "sky130_fd_sc_hd"

set ::env(DESIGN_NAME) dpu_2core
set ::env(VERILOG_FILES) "\
    src/dpu_2core_sv2v.v"
set ::env(CLOCK_PORT) "clk"
set ::env(CLOCK_NET) "clk"
set ::env(CLOCK_PERIOD) 100
set ::env(CLOCK_TREE_SYNTH) 1
set ::env(FP_SIZING) absolute
set ::env(DIE_AREA) "0 0 2000 1700"

set ::env(DESIGN_IS_CORE) 0      
set ::env(SYNTH_MAX_FANOUT) 30

# SRAM config
set ::env(VDD_NETS) "vccd1"
set ::env(GND_NETS) "vssd1"
set ::env(FP_PDN_MACRO_HOOKS) "\
  FULL_DESIGN_WO_PERIPHERY_INS.GLOBAL_MEM_INS.mem_loop\[0\].SP_MEM_MODEL_INS.mem_32x256.sram                                                   vccd1 vssd1 vccd1 vssd1,\
  FULL_DESIGN_WO_PERIPHERY_INS.GLOBAL_MEM_INS.mem_loop\[1\].SP_MEM_MODEL_INS.mem_32x256.sram                                                   vccd1 vssd1 vccd1 vssd1,\
  FULL_DESIGN_WO_PERIPHERY_INS.PROCESSING_LOGIC_INS.pe_loop\[0\].PE_TOP_INS.SP_MEM_MODEL_INS.mem_32x256.sram                                   vccd1 vssd1 vccd1 vssd1,\
  FULL_DESIGN_WO_PERIPHERY_INS.PROCESSING_LOGIC_INS.pe_loop\[1\].PE_TOP_INS.SP_MEM_MODEL_INS.mem_32x256.sram                                   vccd1 vssd1 vccd1 vssd1,\
  FULL_DESIGN_WO_PERIPHERY_INS.STREAMS_TOP_INS.MULTIPLE_STREAMS_INSTR_INS.stream_loop\[0\].STREAM_INS.SP_MEM_MODEL_INS.genblk1.mem_24x128.sram vccd1 vssd1 vccd1 vssd1,\
  FULL_DESIGN_WO_PERIPHERY_INS.STREAMS_TOP_INS.MULTIPLE_STREAMS_LD_INS.stream_loop\[0\].STREAM_INS.SP_MEM_MODEL_INS.genblk1.mem_24x128.sram    vccd1 vssd1 vccd1 vssd1,\
  FULL_DESIGN_WO_PERIPHERY_INS.STREAMS_TOP_INS.MULTIPLE_STREAMS_ST_INS.stream_loop\[0\].STREAM_INS.SP_MEM_MODEL_INS.genblk1.mem_24x128.sram    vccd1 vssd1 vccd1 vssd1,\
  FULL_DESIGN_WO_PERIPHERY_INS.STREAMS_TOP_INS.MULTIPLE_STREAMS_ST_INS.stream_loop\[1\].STREAM_INS.SP_MEM_MODEL_INS.genblk1.mem_24x128.sram    vccd1 vssd1 vccd1 vssd1,\
  FULL_DESIGN_WO_PERIPHERY_INS.STREAMS_TOP_INS.MULTIPLE_STREAMS_LD_INS.stream_loop\[1\].STREAM_INS.SP_MEM_MODEL_INS.genblk1.mem_24x128.sram    vccd1 vssd1 vccd1 vssd1,\
  FULL_DESIGN_WO_PERIPHERY_INS.STREAMS_TOP_INS.MULTIPLE_STREAMS_INSTR_INS.stream_loop\[1\].STREAM_INS.SP_MEM_MODEL_INS.genblk1.mem_24x128.sram vccd1 vssd1 vccd1 vssd1"

set ::env(MACRO_PLACEMENT_CFG) macro_placement.cfg
set ::env(EXTRA_LEFS)      [list $::env(OPENRAM_ROOT)/sky130_sram_1rw_32x256_32.lef $::env(OPENRAM_ROOT)/sky130_sram_1rw_24x128_24.lef]
set ::env(EXTRA_GDS_FILES) [list $::env(OPENRAM_ROOT)/sky130_sram_1rw_32x256_32.gds $::env(OPENRAM_ROOT)/sky130_sram_1rw_24x128_24.gds]
set ::env(EXTRA_LIBS)      [list $::env(OPENRAM_ROOT)/sky130_sram_1rw_32x256_32_TT_1p8V_25C.lib $::env(OPENRAM_ROOT)/sky130_sram_1rw_24x128_24_TT_1p8V_25C.lib]
set ::env(RUN_KLAYOUT_XOR) false
set ::env(MAGIC_DRC_USE_GDS) false
set ::env(QUIT_ON_MAGIC_DRC) false        
        
#set ::env(PL_TARGET_DENSITY) 0.5
#set ::env(FP_CORE_UTIL) 60

set ::env(ROUTING_CORES) 8
#set ::env(FP_PDN_HORIZONTAL_HALO) 6
#set ::env(FP_PDN_VERTICAL_HALO) 6
set ::env(DIODE_INSERTION_STRATEGY) 3

# disable klayout because of https://github.com/hdl/conda-eda/issues/175
set ::env(RUN_KLAYOUT) 0
# disable CVC because of https://github.com/hdl/conda-eda/issues/174
set ::env(RUN_CVC) 0



Overwriting config.tcl


### Launch OpenLANE
The entire flow can take up to 5 hours.

In [13]:
!flow.tcl -design . -ignore_mismatches -tag run1

OpenLane 2022.11.12_3_g1298859-conda
All rights reserved. (c) 2020-2022 Efabless Corporation and contributors.
Available under the Apache License, version 2.0. See the LICENSE file for more details.

[WARNING]: OpenLane may not function properly: not enough values to unpack (expected 3, got 1)
The version of open_pdks used in building the PDK does not match the version OpenLane was tested on (installed: a519523b0d9bc913a6f87a5eed083597ed9e2e93, tested: 0059588eebfc704681dc2368bd1d33d96281d10f)
This may introduce some issues. You may want to re-install the PDK by invoking `make pdk`.
The version of magic used in building the PDK does not match the version OpenLane was tested on (installed: f0668447616c071f145d3d3fa1dfde9762b62b7b, tested: 94daf986ab9aa94a9ae2ac3539fa5def9bd2a1ac)
This may introduce some issues. You may want to re-install the PDK by invoking `make pdk`.
[INFO]: Using configuration in 'config.tcl'...
[INFO]: PDK Root: /users/micas/nshah/Downloads/PhD/Academic/Bayesian_Net

### Output GDS
The output GDS is available in the run directory at runs/run1/results/signoff/. The DPU GDS is too big for converstion to svg. Instead, it has to be viewed in a layout editor (like klayout) directly. Some screenshots from klayout:

#### Full GDS:
![Full GDS](images/DPU_GDS_full.png "Full GDS")

#### GDS without DECAP and FILLER cells
![GDS without DECAP and FILLER cells](images/DPU_GDS_wo_fill_decap.png "GDS without DECAP and FILLER cells")

#### Some important final results:

| | |
|:------------:|:-----------:|
|Total die area|2.0 x 1.7 = 3.4 mm<sup>2</sup>|
|Total on-chip SRAM|6.25 KB|
|Target clock frequency|10 MHz|
|Area of std. cells and SRAM| 1.3 mm<sup>2</sup>|
|Die utilization|40 %|


The detailed report from the tool can be seen below:

In [1]:
import pandas as pd
import pathlib

pd.options.display.max_rows = None
final_summary_reports = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(final_summary_reports[-1])
df.transpose()

,0
design,/users/micas/nshah/Downloads/PhD/Academic/Baye...
design_name,dpu_2core
config,run1
flow_status,flow completed
total_runtime,2h54m20s0ms
routed_runtime,2h20m31s0ms
(Cell/mm^2)/Core_Util,27887.647059
DIEAREA_mm^2,3.4
CellPer_mm^2,13943.823529
OpenDP_Util,20.58
